# training using darts library

TL using the same number of IDs is possible

Reshaping does not seem possible.

when the target dataset has more IDs than the source set, we use the same model to predict multiple ids 

the models are trained on the same amount of IDs as the target test set contains 

residuals of the target test set are not considered 

TODO: Fine tuning and saving outputs


In [5]:
from darts import TimeSeries
from darts.utils.losses import *
from darts.models import *
from darts.metrics.metrics import mse, mae, mape

from utils import data_handling, helpers
import config
import torch
import numpy as np
import csv

four_weeks = -24*7*4

# Load datasets

In [2]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()

for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)

# normalize train and use matrics for val and test
electricity_dict["4_weeks_train"] = electricity_dict["train"][four_weeks:,:]
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)
electricity_dict["4_weeks_train"], _ = helpers.custom_standardizer(electricity_dict["4_weeks_train"], train_standardize_dict)

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [3]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
bavaria_dict["4_weeks_train"] = bavaria_dict["train"][four_weeks:,:]

In [6]:
# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
gp_dict["4_weeks_train"] = gp_dict["train"][four_weeks:,:]

# Define and train models

In [7]:
def train_models(source_data_train, source_data_val, epochs=1):
    """
    -Convert train/val data into Darts TimeSeries format
    -Instantiate different models
    -Fit models to dataset

    Return: fitted models
    """
    ts_train_source = TimeSeries.from_values(source_data_train)
    ts_val_source = TimeSeries.from_values(source_data_val)
   
   
    # model specific parameters
    # nhits/nbeats
    NUM_STACKS = 4
    NUM_BLOCKS = 2
    NUM_LAYERS = 2
    LAYER_WIDTH = 256
    COEFFS_DIM = 5

    # Training settings:
    LR = 0.0005
    BATCH_SIZE = 32
    NUM_EPOCHS = epochs

    # shared parameters
    LAYER_NORM = 'LayerNorm'
    USE_REVIN = True
    LOSS_FN = torch.nn.MSELoss()
    IN_LEN = 96
    OUT_LEN = 96
    DROPOUT = 0.2  # slightly higher than "generic" because we want more robustnes for TL
    VERBOSE = False

    # reproducibility
    np.random.seed(42)
    torch.manual_seed(42)

    nhits_model = NHiTSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        activation='ReLU',
        num_stacks=NUM_STACKS, 
        num_blocks=NUM_BLOCKS, 
        num_layers=NUM_LAYERS, 
        layer_widths=LAYER_WIDTH, 
   #     expansion_coefficient_dim=COEFFS_DIM, 
    #    trend_polynomial_degree=2, 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=USE_REVIN,
        pl_trainer_kwargs={
                "enable_progress_bar": True,
                # change this one to "gpu" if your notebook does run in a GPU environment:
                "accelerator": "gpu",},
)

    nbeats_model = NBEATSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        batch_size=BATCH_SIZE,
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        expansion_coefficient_dim=COEFFS_DIM,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=USE_REVIN,
        activation='ReLU',
        optimizer_kwargs={"lr": LR},
        pl_trainer_kwargs={
            "enable_progress_bar": True,
            # change this one to "gpu" if your notebook does run in a GPU environment:
            "accelerator": "gpu",},
    )


    transformer_model = TransformerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN,
        output_chunk_shift=0, 
        d_model=LAYER_WIDTH, 
        nhead=4, 
        num_encoder_layers=3, 
        num_decoder_layers=3, 
        dim_feedforward=LAYER_WIDTH, 
        dropout=DROPOUT, 
        activation='relu', 
        loss_fn=LOSS_FN,
        norm_type=None, 
        custom_encoder=None, 
        custom_decoder=None,
        use_reversible_instance_norm=USE_REVIN,
    )

    tsmixer_model = TSMixerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN, 
        output_chunk_shift=0,
        hidden_size=LAYER_WIDTH, 
        ff_size=LAYER_WIDTH, 
        num_blocks=NUM_BLOCKS, 
        activation='ReLU', 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        norm_type=LAYER_NORM, 
        normalize_before=False, 
        use_static_covariates=False,
        use_reversible_instance_norm=USE_REVIN,

    )

    nhits_model.fit(        
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,)

    nbeats_model.fit(
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,

    )

    transformer_model.fit(
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,

    )

    tsmixer_model.fit(
        ts_train_source,
        val_series=ts_val_source,
        num_loader_workers=4,
        epochs=NUM_EPOCHS,
        max_samples_per_ts=None,
        verbose = VERBOSE,

    )


    return transformer_model, nbeats_model, tsmixer_model, nhits_model



def evaluate(transformer, nbeats, nhits, tsnmix, target_test):
    """
    Evaluates models on target test set
    

    Input:  -trained models
            -List of target test sets shaped according to models

    Output: [transformer, nhits, tsnmix]-Losses
    """

    forecasting_endpoint = int(len(target_test)) - 96*2
    target_test = TimeSeries.from_values(target_test)
    window = [target_test[i:i+96] for i in range(0, forecasting_endpoint, 5)]
    target = [target_test[i+96:i+96+96] for i in range(0, forecasting_endpoint, 5)]

    # predict over dataloader with slidingwindow implementation and 5 time step shifts for each input
    preds_transformer = transformer.predict(n=96, series=window)
    preds_nhits = nhits.predict(n=96, series=window)
    preds_tsnmix = tsnmix.predict(n=96, series=window)
    preds_nbeats = nbeats.predict(n=96, series=window)


    mse_transformer = mse(preds_transformer, target)
    mse_nhits = mse(preds_nhits, target)
    mse_tsmix = mse(preds_tsnmix, target)
    mse_nbeats = mse(preds_nbeats, target)

    length_loss = len(mse_transformer)

    mae_transformer = mae(preds_transformer, target)
    mae_nhits = mae(preds_nhits, target)
    mae_tsmix = mae(preds_tsnmix, target)
    mae_nbeats = mae(preds_nbeats, target)


#    mape_transformer = mape(preds_transformer, target)
 #   mape_nhits = mape(preds_nhits, target)
  #  mape_tsmix = mape(preds_tsnmix, target)
   # mape_nbeats = mape(preds_nbeats, target)



    mse_mae_mape_transformer = sum(mse_transformer) / length_loss, sum(mae_transformer) / length_loss#, sum(mape_transformer) / length_loss
    mse_mae_mape_nhits = sum(mse_nhits) / length_loss, sum(mae_nhits) / length_loss#, sum(mape_nhits) / length_loss
    mse_mae_mape_tsmix = sum(mse_tsmix) / length_loss, sum(mae_tsmix) / length_loss#, sum(mape_tsmix) / length_loss
    mse_mae_mape_nbeats = sum(mse_nbeats) / length_loss, sum(mae_nbeats) / length_loss#, sum(mape_nbeats) / length_loss

    return mse_mae_mape_transformer, mse_mae_mape_nhits, mse_mae_mape_nbeats, mse_mae_mape_tsmix

In [6]:
output_path_full = config.CONFIG_OUTPUT_PATH["darts"] / "electricity_metrics_full.csv"
output_path_short = config.CONFIG_OUTPUT_PATH["darts"] / "electricity_metrics_short.csv"

# train on 4 weeks and predict on electricity dataset
transformer_model, nbeats_model, tsmixer_model, nhits_model = train_models(electricity_dict["4_weeks_train"], electricity_dict["validation"], 10)
electricity_metrics_short = evaluate(transformer_model, nbeats_model, nhits_model, tsmixer_model, electricity_dict["test"])

# Open the file in write mode
with open(output_path_short, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Optionally write headers, if you want to label the columns
    writer.writerow(["mse", "mae"])

    # Write the data
    writer.writerows(electricity_metrics_short)

# full training and evaluation
transformer_model, nbeats_model, tsmixer_model, nhits_model = train_models(electricity_dict["train"], electricity_dict["validation"], 10)
electricity_metrics_full = evaluate(transformer_model, nbeats_model, nhits_model, tsmixer_model, electricity_dict["test"])

# Open the file in write mode
with open(output_path_full, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Optionally write headers, if you want to label the columns
    writer.writerow(["mse", "mae"])

    # Write the data
    writer.writerows(electricity_metrics_full)

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/torch/random.py:107: UserWarning: CUDA reports that you have 3 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of GPUs.  If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using.  For example, if you are using CPU only, set CUDA_VISIBLE_DEVICES= or devices=[]; if you are using GPU 0 only, set CUDA_VISIBLE_DEVICES=0 or devices=[0].  To initialize all devices and suppress this warning, set the 'devices' keyword argument to `range(torch.cuda.device_count())`.
  warnings.warn(
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3

Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 16.82it/s]

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 21.74it/s]


...

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 19.33it/s]

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:00<00:00, 18.53it/s]


In [8]:
output_path_full = config.CONFIG_OUTPUT_PATH["darts"] / "bavaria_metrics_full.csv"
output_path_short = config.CONFIG_OUTPUT_PATH["darts"] / "bavaria_metrics_short.csv"

# train on 4 weeks dataset and evaluate
transformer_model, nbeats_model, tsmixer_model, nhits_model = train_models(bavaria_dict["4_weeks_train"], bavaria_dict["validation"], 10)
bavaria_metrics_short = evaluate(transformer_model, nbeats_model, nhits_model, tsmixer_model, bavaria_dict["test"])

# Open the file in write mode
with open(output_path_short, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Optionally write headers, if you want to label the columns
    writer.writerow(["mse", "mae"])

    # Write the data
    writer.writerows(bavaria_metrics_short)


# train and predict on bavaria dataset
transformer_model, nbeats_model, tsmixer_model, nhits_model = train_models(bavaria_dict["train"], bavaria_dict["validation"], 10)
bavaria_metrics_long = evaluate(transformer_model, nbeats_model, nhits_model, tsmixer_model, bavaria_dict["test"])



# Open the file in write mode
with open(output_path_full, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Optionally write headers, if you want to label the columns
    writer.writerow(["mse", "mae"])

    # Write the data
    writer.writerows(bavaria_metrics_long)

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


`Trainer.fit` stopped: `max_epochs=10` reached.
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
`Trainer.fit` stopped: `max_epochs=10` reached.
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0

Predicting DataLoader 0: 100%|██████████| 17/17 [00:01<00:00, 13.50it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 20.49it/s]


...

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:01<00:00, 16.53it/s]

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 24.06it/s]


In [10]:
output_path_full = config.CONFIG_OUTPUT_PATH["darts"] / "gp_metrics_full.csv"
output_path_short = config.CONFIG_OUTPUT_PATH["darts"] / "gp_metrics_short.csv"


# train and predict on genome project 2 dataset
transformer_model, nbeats_model, tsmixer_model, nhits_model = train_models(gp_dict["4_weeks_train"], gp_dict["validation"], 10)
gp_metrics_short = evaluate(transformer_model, nbeats_model, nhits_model, tsmixer_model, gp_dict["test"])

# Open the file in write mode
with open(output_path_short, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Optionally write headers, if you want to label the columns
    writer.writerow(["mse", "mae"])

    # Write the data
    writer.writerows(gp_metrics_short)

    
# train and predict on genome project 2 dataset
transformer_model, nbeats_model, tsmixer_model, nhits_model = train_models(gp_dict["train"], gp_dict["validation"], 10)
gp_metrics_long = evaluate(transformer_model, nbeats_model, nhits_model, tsmixer_model, gp_dict["test"])

# Open the file in write mode
with open(output_path_full, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Optionally write headers, if you want to label the columns
    writer.writerow(["mse", "mae"])

    # Write the data
    writer.writerows(gp_metrics_long)



Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
`Trainer.fit` stopped: `max_epochs=20` reached.
Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may t

Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 11.63it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 11.91it/s]


Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:37<00:00,  0.48it/s]

Trainer will use only 1 of 3 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=3)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 13.30it/s]
